In [1]:
import re
import os
import time
import datetime
import random
import requests
import numpy as np
import pandas as pd
import ws_func
from datetime import datetime
from os import system   
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Scrape CNN Articles

In [2]:
ws_func.scrape_cnn(start=(2019, 12, 1), stop=(2020, 1, 1))

All URLS in date range scraped
Number of article URLs: 3

